# Using Hand Crafted Features

### Importing necesssary libraries

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score

The images from the dataset are read

In [2]:
# Paths to dataset
dataset_path = r"dataset"  # Using raw strings bacause the file names contain backslashes and escape characters
mask_path = os.path.join(dataset_path, "with_mask")
no_mask_path = os.path.join(dataset_path, "without_mask")

In [3]:
#loading a sample image to show that images with 0_0_≈˙◊¢ are not loading correctly
sample_image = cv2.imread('classification_dataset\with_mask\0_0_≈˙◊¢ 2020-02-23 132115.png')
if sample_image is None:
    print('Image not loaded correctly')


Image not loaded correctly


<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_26472\2602037394.py:2: SyntaxWarning: invalid escape sequence '\w'
  sample_image = cv2.imread('classification_dataset\with_mask\0_0_≈˙◊¢ 2020-02-23 132115.png')


This error shows that the images with this kind of a name are not being loaded. Manual inspection shows that these kind of images only exist in the with_mask category.

In [4]:
# Now we create a python script to rename all the wrongly names images
import os

# Define the folder path where images are stored
folder_path = mask_path

# Define the characters to be replaced
special_chars = "≈˙◊¢"

# List all files in the folder
for filename in os.listdir(folder_path):
    old_path = os.path.join(folder_path, filename)

    # Replace specific special characters with '_'
    new_filename = filename
    for char in special_chars:
        new_filename = new_filename.replace(char, "_")

    new_path = os.path.join(folder_path, new_filename)

    # Rename the file if necessary
    if old_path != new_path:
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} → {new_filename}")



Now the images can be loaded without issues

In [5]:
# Feature extraction functions
def ExtractHogFeatures(img):
    grayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hogFeatures, _ = hog(grayImg, pixels_per_cell=(8, 8), cells_per_block=(2, 2), 
                         block_norm='L2-Hys', visualize=True)
    return hogFeatures

def ExtractLbpFeatures(img):
    grayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lbpImage = local_binary_pattern(grayImg, P=8, R=1, method="uniform")
    histValues, _ = np.histogram(lbpImage.ravel(), bins=np.arange(0, 10), range=(0, 10))
    histValues = histValues.astype("float")
    histValues /= histValues.sum()
    return histValues

def ExtractColorHistogram(img, bins=(8, 8, 8)):
    hsvImg = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    colorHist = cv2.calcHist([hsvImg], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    colorHist = cv2.normalize(colorHist, colorHist).flatten()
    return colorHist


Extracting features and storing them in numpy arrays

In [6]:
# Load dataset and extract features
FeatureList, LabelList = [], []

for ClassLabel, ClassPath in enumerate([mask_path, no_mask_path]):  # 0: with_mask, 1: without_mask
    for FileName in os.listdir(ClassPath):
        ImgPath = r"{}".format(os.path.join(ClassPath, FileName))  # Use raw string path
        InputImage = cv2.imread(ImgPath)
        if InputImage is not None:
            InputImage = cv2.resize(InputImage, (128, 128))
            # Extract features
            HogFeat = ExtractHogFeatures(InputImage)
            LbpFeat = ExtractLbpFeatures(InputImage)
            ColorFeat = ExtractColorHistogram(InputImage)
            # Combine features
            CombinedFeatures = np.hstack([HogFeat, LbpFeat, ColorFeat])
            FeatureList.append(CombinedFeatures)
            LabelList.append(ClassLabel)
        else:
            print(f"Error loading image: {ImgPath}")

# Convert to NumPy arrays
X = np.array(FeatureList)
y = np.array(LabelList)


Error loading image: dataset\with_mask\0_0_œ¬‘ÿ.png


### Data is split into 3 parts - Train (70%), Validation (15%), Test (15%)

In [7]:
# **Simplified Train-Validation-Test Split (70%-15%-15%)**
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

Creating and training the three ML classifiers (Using hand crafted features)
- Random Forest
- SVM
- XGBoost

In [8]:
# Train Random Forest classifier
RandomForest = RandomForestClassifier(n_estimators=100, random_state=42)
RandomForest.fit(X_train, y_train)
RfValPreds = RandomForest.predict(X_val)
RfF1Score = f1_score(y_val, RfValPreds)
print(f"Random Forest F1 Score: {RfF1Score:.4f}")
print("Classification Report (Random Forest):")
print(classification_report(y_val, RfValPreds))


Random Forest F1 Score: 0.9228
Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       334
           1       0.95      0.90      0.92       280

    accuracy                           0.93       614
   macro avg       0.93      0.93      0.93       614
weighted avg       0.93      0.93      0.93       614



In [9]:
# Train SVM classifier
SvmClassifier = SVC(kernel='linear', C=1.0, random_state=42)
SvmClassifier.fit(X_train, y_train)
SvmValPreds = SvmClassifier.predict(X_val)
SvmF1Score = f1_score(y_val, SvmValPreds)
print(f"SVM F1 Score: {SvmF1Score:.4f}")
print("Classification Report (SVM):")
print(classification_report(y_val, SvmValPreds))


SVM F1 Score: 0.9250
Classification Report (SVM):
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       334
           1       0.93      0.93      0.93       280

    accuracy                           0.93       614
   macro avg       0.93      0.93      0.93       614
weighted avg       0.93      0.93      0.93       614



In [10]:
# Train XGBoost Classifier
XgbClassifier = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
XgbClassifier.fit(X_train, y_train)
XgbValPreds = XgbClassifier.predict(X_val)
XgbF1Score = f1_score(y_val, XgbValPreds)
print(f"XGBoost F1 Score: {XgbF1Score:.4f}")
print("Classification Report (XGBoost):")
print(classification_report(y_val, XgbValPreds))


XGBoost F1 Score: 0.9236
Classification Report (XGBoost):
              precision    recall  f1-score   support

           0       0.92      0.95      0.94       334
           1       0.94      0.91      0.92       280

    accuracy                           0.93       614
   macro avg       0.93      0.93      0.93       614
weighted avg       0.93      0.93      0.93       614



Scores are calculated and the best model is written as output:

In [11]:
# Compare validation F1-scores of all models
ModelScores = {
    "SVM": (SvmClassifier, SvmF1Score),
    "Random Forest": (RandomForest, RfF1Score),
    "XGBoost": (XgbClassifier, XgbF1Score)
}

# Print individual validation F1-scores
print("Validation F1-Scores:")
for modelName, (_, f1ScoreVal) in ModelScores.items():
    print(f"  - {modelName}: {f1ScoreVal:.4f}")

# Choose best model based on validation F1-score
BestModelName, (BestModel, BestF1Score) = max(ModelScores.items(), key=lambda x: x[1][1])

# Final testing on the best model
FinalTestPreds = BestModel.predict(X_test)
FinalTestF1 = f1_score(y_test, FinalTestPreds)
FinalTestReport = classification_report(y_test, FinalTestPreds)

# Print results
print(f"\nBest Model: {BestModelName} (Validation F1-Score = {BestF1Score:.4f})")
print(f"Test F1-Score for Best Model ({BestModelName}): {FinalTestF1:.4f}")
print("Classification Report (Test Set):")
print(FinalTestReport)


Validation F1-Scores:
  - SVM: 0.9250
  - Random Forest: 0.9228
  - XGBoost: 0.9236

Best Model: SVM (Validation F1-Score = 0.9250)
Test F1-Score for Best Model (SVM): 0.8917
Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.91      0.92      0.91       336
           1       0.90      0.89      0.89       279

    accuracy                           0.90       615
   macro avg       0.90      0.90      0.90       615
weighted avg       0.90      0.90      0.90       615



In summary, out of the 3 chosen ML classifiers, SVM performs the best at classifying the image dataset using hand crafted features

# Using CNN(Automatic Feature Learning)

#### Preprocessing involves renaming file names uniformly accross both datasets

In [ ]:
def rename(dataset_path,string):
    # Get all image files
    image_files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]
    
    # Rename files sequentially
    for index, filename in enumerate(image_files, start=1):
        old_path = os.path.join(dataset_path, filename)

        # Extract the file extension (e.g., .jpg, .png)
        extension = os.path.splitext(filename)[1]  # Includes the dot

        # Generate new filename
        new_filename = f"image_{string}_{index}{extension}"
        new_path = os.path.join(dataset_path, new_filename)

        # Rename the file
        os.rename(old_path, new_path)
        # print(f"Renamed: {filename} → {new_filename}")


rename(r"dataset/with_mask", 'with_mask')
rename(r"dataset/without_mask", 'without_mask')

print(" All images renamed")



### Importing libraries

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
from torch.optim import Adam, RMSprop
from tqdm import tqdm

- Setting up GPU (this was run on Kaggle for convenience and faster results)
- Setting up the hyperparameters

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}, GPUs: {torch.cuda.device_count()}")

dataset_path = "/kaggle/input/dataset"
img_size = (128, 128)
batch_sizes = [16, 32]
learning_rates = [0.01, 0.001, 0.0001]
optimizers = [Adam, RMSprop]
activations = ['relu', 'tanh']

Created 2 CNN classes - one for ReLU and one for Tanh activation functions, we were facing some issue in passing it as a parameter to the constructor of this class

In [ ]:
class CNNReLU(nn.Module):
    def __init__(self):
        super(CNNReLU, self).__init__()
        act = nn.ReLU
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), act(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), act(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), act(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 128), act(),
            nn.Linear(128, 1), nn.Sigmoid()
        )
    def forward(self, x): return self.model(x)


class CNNTanh(nn.Module):
    def __init__(self):
        super(CNNTanh, self).__init__()
        act = nn.Tanh
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), act(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), act(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), act(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 128), act(),
            nn.Linear(128, 1), nn.Sigmoid()
        )
    def forward(self, x): return self.model(x)

### Data is split into 3 parts - Train (70%), Validation (15%), Test (15%)

In [ ]:
transform = transforms.Compose([transforms.Resize(img_size), transforms.ToTensor()])
full_dataset = datasets.ImageFolder(dataset_path, transform=transform)

train_len = int(0.7 * len(full_dataset))
val_len = int(0.15 * len(full_dataset))
test_len = len(full_dataset) - train_len - val_len
train_set, val_set, test_set = random_split(full_dataset, [train_len, val_len, test_len])

### Created function to create model, run and calculate loss

In [ ]:
def train_and_evaluate(model, train_loader, val_loader, test_loader, optimizer, epochs=15):
    criterion = nn.BCELoss()
    model = model.to(device)

    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False):
            x, y = x.to(device), y.float().unsqueeze(1).to(device)
            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} Avg Loss: {avg_loss:.4f}")

    # Evaluation
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            outputs = model(x)
            preds = (outputs.cpu().numpy() > 0.5).astype(int)
            y_pred.extend(preds.flatten())
            y_true.extend(y.numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return acc, f1

## Final output cell, calls function in a loop and trains model for each parameter combination. It stores result in a list which is later used to compare and find the best combination.

In [ ]:
all_results = []

for lr in learning_rates:
    for batch_size in batch_sizes:
        for opt_class in optimizers:
            for act in activations:
                print(f"\n🔁 Training with LR={lr}, Batch={batch_size}, Optimizer={opt_class.__name__}, Activation={act}")

                if act == 'relu':
                    model = CNNReLU()
                elif act == 'tanh':
                    model = CNNTanh()
                else:
                    continue

                train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(val_set, batch_size=batch_size)
                test_loader = DataLoader(test_set, batch_size=batch_size)

                optimizer = opt_class(model.parameters(), lr=lr)

                acc, f1 = train_and_evaluate(model, train_loader, val_loader, test_loader, optimizer)

                all_results.append({
                    "Learning Rate": lr,
                    "Batch Size": batch_size,
                    "Optimizer": opt_class.__name__,
                    "Activation": act,
                    "Test Accuracy": acc,
                    "Test F1-Score": f1
                })

### Summary of result and comparative overview of the same

In [ ]:
results_df = pd.DataFrame(all_results)
results_df.sort_values(by="Test F1-Score", ascending=False, inplace=True)
print(results_df.head())

best_model = results_df.iloc[0]
print("\nBest CNN Configuration:")
for col in best_model.index:
    print(f"{col}: {best_model[col]}")

In conclusion, the CNN based classifier outperforms the ML based classifier by a significant margin, as shown by the accuracy scores calculated by testing on the same dataset